# LouieAI Error Handling

This notebook demonstrates error handling patterns specific to the LouieAI `lui` interface.

## Setup

In [1]:
import os

import graphistry
import pandas as pd

from louieai import louie

# Authenticate
g = graphistry.register(
    api=3,
    server=os.environ.get("GRAPHISTRY_SERVER", "hub.graphistry.com"),
    username=os.environ.get("GRAPHISTRY_USERNAME"),
    password=os.environ.get("GRAPHISTRY_PASSWORD"),
)

# Create LouieAI interface
lui = louie(g, server_url=os.environ.get("LOUIE_SERVER", "https://den.louie.ai"))
print("✅ LouieAI ready")

✅ LouieAI ready


## Handling Missing Data

In [2]:
# Safe pattern for checking if data exists
lui("What is 2+2?")  # Text-only response, no DataFrame

# Always check if DataFrame exists before using it
if lui.df is not None:
    print(f"DataFrame shape: {lui.df.shape}")
    print(lui.df.head())
else:
    print("No DataFrame returned (text-only response)")
    print(f"Text response: {lui.text}")

No DataFrame returned (text-only response)
Text response: The question is a simple arithmetic problem.



## Checking for Errors

In [3]:
# Check if the last query had errors
if lui.has_errors:
    print("⚠️ Last query had errors:")
    for error in lui.errors:
        print(f"  - {error}")
else:
    print("✅ No errors in last query")

✅ No errors in last query


## Using History for Recovery

In [4]:
# Make several queries
lui("Create a DataFrame with sales data", pd.DataFrame({"sales": [100, 200, 300]}))
lui("What is the weather today?")  # No DataFrame

# Access previous responses
current_df = lui.df  # May be None
previous_df = lui[-2].df if len(lui._history) >= 2 else None

# Use the most recent DataFrame available
df = current_df if current_df is not None else previous_df
if df is not None:
    print(f"Using DataFrame with shape: {df.shape}")
else:
    print("No DataFrame available in recent history")

Using DataFrame with shape: (3, 1)


## Retry Pattern

In [5]:
def query_with_retry(prompt, data=None, max_retries=3):
    """Query LouieAI with automatic retry on failure."""
    for attempt in range(max_retries):
        try:
            lui(prompt, data)

            if not lui.has_errors:
                print(f"✅ Success on attempt {attempt + 1}")
                return lui.text, lui.df

            print(f"⚠️ Attempt {attempt + 1} had errors, retrying...")

        except Exception as e:
            if attempt < max_retries - 1:
                print(f"Attempt {attempt + 1} failed: {e}")
                continue
            else:
                raise

    return None, None


# Example usage
text, df = query_with_retry("Analyze this data", pd.DataFrame({"value": [1, 2, 3]}))

✅ Success on attempt 1


## Using Traces for Debugging

In [6]:
# Enable traces to see reasoning steps
lui.traces = True

# Make a query - will show detailed reasoning
lui("Calculate the average of [10, 20, 30, 40, 50]")

print(f"Result: {lui.text}")

# Disable traces for normal operation
lui.traces = False

Result: To calculate the average of a list of numbers, I need to sum the numbers and then divide by the count of the numbers.

Plan:
1. Sum the numbers: 10 + 20 + 30 + 40 + 50.
2. Count the numbers: There are 5 numbers.
3. Divide the sum by the count to get the average.

Calculation:
1. Sum = 10 + 20 + 30 + 40 + 50 = 150
2. Count = 5
3. Average = Sum / Count = 150 / 5 = 30

The average of [10, 20, 30


## Batch Processing with Error Tracking

In [7]:
# Process multiple DataFrames with error tracking
datasets = [
    pd.DataFrame({"Q1": [100, 150, 200]}),
    pd.DataFrame({"Q2": [120, 160, 210]}),
    pd.DataFrame({"Q3": [130, 170, 220]}),
    pd.DataFrame({"Q4": [140, 180, 230]}),
]

results = []
errors = []

for i, df in enumerate(datasets, 1):
    try:
        lui(f"Calculate the total for quarter Q{i}", df)

        if lui.df is not None:
            results.append({"quarter": f"Q{i}", "response": lui.text, "data": lui.df})
        else:
            errors.append({"quarter": f"Q{i}", "error": "No DataFrame returned"})

    except Exception as e:
        errors.append({"quarter": f"Q{i}", "error": str(e)})

print("\n📊 Batch Summary:")
print(f"Successful: {len(results)}")
print(f"Failed: {len(errors)}")


📊 Batch Summary:
Successful: 4
Failed: 0


## Safe Access Patterns

In [8]:
# Pattern 1: Defensive checking
def get_dataframe_safely():
    """Get DataFrame with fallback to empty."""
    return lui.df if lui.df is not None else pd.DataFrame()


# Pattern 2: Check multiple attributes
def get_all_results():
    """Get all available results from last query."""
    results = {
        "text": lui.text,
        "has_dataframe": lui.df is not None,
        "has_errors": lui.has_errors,
    }

    if lui.df is not None:
        results["df_shape"] = lui.df.shape
        results["df_id"] = lui.df_id

    return results


# Example usage
lui("Generate sample data")
results = get_all_results()
print("Query results:", results)

Query results: {'text': "Successfully generated a sample dataframe with 100 rows, containing columns for 'Name', 'Email', 'Address', and 'Phone Number'. The dataset was created using the Faker library, ensuring realistic and varied data entries. This process was executed efficiently, with the entire operation completed in approximately 0.3 seconds. The resulting dataframe has been saved as an artifact named 'sample_data', ready for further use or analysis.", 'has_dataframe': False, 'has_errors': False}


/home/lmeyerov/Work/louie-py/src/louieai/auth.py:209: RuntimeWarning: Failed to fetch dataframe B_crQe5JNI from thread D_xsOfLperiNkIYgT_O5bK. URL: http://localhost:8000/api/dthread/D_xsOfLperiNkIYgT_O5bK/df/block/B_crQe5JNI/arrow. Error: HTTPStatusError: Client error '404 Not Found' for url 'http://localhost:8000/api/dthread/D_xsOfLperiNkIYgT_O5bK/df/block/B_crQe5JNI/arrow'
For more information check: https://developer.mozilla.org/en-US/docs/Web/HTTP/Status/404
  return func(self, *args, **kwargs)


## Summary

Key error handling patterns for LouieAI:

1. **Always check `lui.df is not None`** before using DataFrames
2. **Use `lui.has_errors`** to check for query errors
3. **Access history with `lui[-1]`, `lui[-2]`** for recovery
4. **Enable `lui.traces = True`** for debugging
5. **Implement retry logic** for critical queries
6. **Use defensive patterns** with fallbacks to empty DataFrames